In [1]:
import math
import numpy as np
import wandb
import pickle
import os
import shapely.wkt as wkt
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from torch_geometric.transforms import LineGraph

import gzip
import xml.etree.ElementTree as ET

import torch
import torch_geometric
from torch_geometric.data import Data

import processing_io as pio
import sys
import os
import joblib
import json

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from shapely.geometry import Point, LineString, box
from matplotlib.colors import TwoSlopeNorm

from shapely.ops import unary_union
from mpl_toolkits.axes_grid1 import make_axes_locatable
from torch_geometric.data import Data, Batch
import torch
from torch_geometric.data import Data
import help_functions as hf

districts = gpd.read_file("../../data/visualisation/districts_paris.geojson")

# Add the 'scripts' directory to the Python path
scripts_path = os.path.abspath(os.path.join('..'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

import gnn_io as gio
import gnn_architectures as garch

highway_mapping = {
    'trunk': 0, 'trunk_link': 0, 'motorway_link': 0,
    'primary': 1, 'primary_link': 1,
    'secondary': 2, 'secondary_link': 2,
    'tertiary': 3, 'tertiary_link': 3,
    'residential': 4, 'living_street': 5,
    'pedestrian': 6, 'service': 7,
    'construction': 8, 'unclassified': 9,
    'np.nan': -1
}

def compute_r2_torch_with_mean_targets(mean_targets, preds, targets):
    ss_tot = torch.sum((targets - mean_targets) ** 2)
    ss_res = torch.sum((targets - preds) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

def validate_one_model(model, data, loss_func, device):
    model.eval()
    pred = []
    actual = []
    with torch.inference_mode():
        input_node_features, targets = data.x.to(device), data.y.to(device)
        predicted = model(data.to(device))
        # print(predicted.shape)
        pred.append(predicted)
        actual.append(targets)
        val_loss = loss_func(predicted, targets).item()
    actual_vals = torch.cat(actual)
    predicted_vals = torch.cat(pred)
    
    mean_targets = torch.mean(targets)
    r_squared = compute_r2_torch_with_mean_targets(mean_targets = mean_targets, preds=predicted_vals, targets=actual_vals)
    baseline_loss = loss_func(targets, torch.full_like(predicted_vals, mean_targets))
    return val_loss, r_squared, targets, predicted, baseline_loss


import math
import numpy as np
import wandb
import pickle
import os
import shapely.wkt as wkt
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from torch_geometric.transforms import LineGraph

import gzip
import xml.etree.ElementTree as ET

import torch
import torch_geometric
from torch_geometric.data import Data

import processing_io as pio
import sys
import os
import joblib
import json

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from shapely.geometry import Point, LineString, box
from matplotlib.colors import TwoSlopeNorm

from shapely.ops import unary_union
from mpl_toolkits.axes_grid1 import make_axes_locatable
from torch_geometric.data import Data, Batch
import torch
from torch_geometric.data import Data
import alphashape
from matplotlib.lines import Line2D
from shapely.geometry import Polygon


In [2]:
# Parameters to define
run_path = '/home/enatterer/Development/gnn_predicting_effects_of_traffic_policies/data/runs_optimized/pnc_local_[256]_pnc_global_[512_256]_hidden_layer_str_[512_512_256_128_64]_dropout_0.3_use_dropout_False/'
point_net_conv_layer_structure_local_mlp = [256]
point_net_conv_layer_structure_global_mlp = [512,256]
gat_conv_layer_structure = [512,512,256,128,64]
dropout = 0.3
use_dropout = False 
in_channels = 6 
out_channels = 1 

model_path = run_path +  'trained_model/model.pth'
data_created_during_training = run_path + 'data_created_during_training/'
indices_of_datasets_to_use = [0, 1, 3, 4]

scaler_x = joblib.load(data_created_during_training + 'x_scaler.pkl')
scaler_pos = joblib.load(data_created_during_training + 'pos_scaler.pkl')

# Initialize the model
model = garch.MyGnn(in_channels=in_channels, out_channels=out_channels, 
                    point_net_conv_layer_structure_local_mlp=point_net_conv_layer_structure_local_mlp, 
                    point_net_conv_layer_structure_global_mlp = point_net_conv_layer_structure_global_mlp,
                    gat_conv_layer_structure=gat_conv_layer_structure,
                    dropout=dropout,
                    use_dropout=use_dropout)

# Load the model state dictionary
model.load_state_dict(torch.load(model_path))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Initializing PointNetConv(local_nn=Sequential(
  (0): Linear(in_features=6, out_features=256, bias=True)
  (1): ReLU()
), global_nn=Sequential(
  (0): Linear(in_features=256, out_features=512, bias=True)
  (1): Linear(in_features=512, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=512, bias=True)
  (4): ReLU()
))
Initializing 0.weight with kaiming_normal
Initializing 0.bias with zeros
Initializing 0.weight with kaiming_normal
Initializing 0.bias with zeros
Initializing 1.weight with kaiming_normal
Initializing 1.bias with zeros
Initializing 3.weight with kaiming_normal
Initializing 3.bias with zeros
Initializing Linear(in_features=6, out_features=256, bias=True)
Initializing Linear(in_features=256, out_features=512, bias=True)
Initializing Linear(in_features=512, out_features=256, bias=True)
Initializing Linear(in_features=256, out_features=512, bias=True)
Initializing GATConv(512, 512, heads=1)
Initializing GATConv(512, 256, heads=1)
Initializin

In [3]:
# Load the test dataset created during training
test_set_dl = torch.load(data_created_during_training + 'test_dl.pt')

# Load the DataLoader parameters
with open(data_created_during_training + 'test_loader_params.json', 'r') as f:
    test_set_dl_loader_params = json.load(f)
    
# Remove or correct collate_fn if it is incorrectly specified
if 'collate_fn' in test_set_dl_loader_params and isinstance(test_set_dl_loader_params['collate_fn'], str):
    del test_set_dl_loader_params['collate_fn']  # Remove it to use the default collate function
    
test_set_loader = torch.utils.data.DataLoader(test_set_dl, **test_set_dl_loader_params)

In [4]:
test_data = "../../data/test_data/gdf_pop_1pm_policy_in_1_2_3_4.geojson"
test_data = gpd.read_file(test_data)
base_case = "../../data/test_data/gdf_basecase_mean_pop_1pm.geojson"
base_case = gpd.read_file(base_case)

In [5]:
i = 59
loss_fct= torch.nn.MSELoss()
test_loss, r_squared, actual_vals, predictions, baseline_loss = validate_one_model(model, test_set_loader.dataset[i], loss_fct, device)

gdf = hf.data_to_geodataframe(data=test_set_loader.dataset[0], original_gdf=test_data, predicted_values=predictions)
gdf_with_og_values = hf.map_to_original_values(input_gdf=gdf, scaler_x=scaler_x)
gdf_with_og_values.crs = "EPSG:2154"
gdf_with_og_values.to_crs("EPSG:4326", inplace=True)

In [6]:
indices_roads_with_highway_primary_sec_tertiary = gdf_with_og_values[gdf_with_og_values['og_highway'].isin([1,2,3])].index
indices_roads_with_highway_primary_ = gdf_with_og_values[gdf_with_og_values['og_highway'].isin([1])].index
indices_roads_with_highway_sec = gdf_with_og_values[gdf_with_og_values['og_highway'].isin([2])].index
indices_roads_with_highway_tertiary = gdf_with_og_values[gdf_with_og_values['og_highway'].isin([3])].index

indices_roads_with_highway_not_primary_sec_tertiary = gdf_with_og_values[~gdf_with_og_values['og_highway'].isin([1, 2, 3])].index

gdf_with_og_values['og_capacity_reduction_rounded'] = gdf_with_og_values['og_capacity_reduction'].round(decimals=3)
tolerance = 1e-3
indices_roads_with_cap_reduction = gdf_with_og_values[gdf_with_og_values['og_capacity_reduction_rounded'] < -1e-3].index
indices_roads_with_no_cap_reduction = gdf_with_og_values[gdf_with_og_values['og_capacity_reduction_rounded'] >= -1e-3].index

indices_roads_with_highway_primary_sec_tertiary_and_cap_reduction = gdf_with_og_values[
    (gdf_with_og_values['og_highway'].isin([1, 2, 3])) & 
    (gdf_with_og_values['og_capacity_reduction_rounded'] < -1e-3)
].index
indices_roads_with_highway_primary_sec_tertiary_and_not_cap_reduction = gdf_with_og_values[
    (gdf_with_og_values['og_highway'].isin([1, 2, 3])) & 
    (gdf_with_og_values['og_capacity_reduction_rounded'] >= -1e-3)
].index

no_filter = gdf_with_og_values.index

In [7]:
districts_of_interest = [1, 2, 3, 4, 6, 8, 9, 11, 14, 20]
districts = gpd.read_file("../../data/visualisation/districts_paris.geojson")
target_districts = districts[districts['c_ar'].isin(districts_of_interest)]
gdf_with_og_values['intersects_target_districts'] = gdf_with_og_values.apply(lambda row: target_districts.intersects(row.geometry).any(), axis=1)

In [8]:
indices_this_zone = gdf_with_og_values[gdf_with_og_values['intersects_target_districts']].index
overlap = indices_this_zone.intersection(indices_roads_with_highway_primary_sec_tertiary)

indices_to_filter_for = overlap

filtered_actual = actual_vals[indices_to_filter_for]
filtered_actual_mean = torch.mean(filtered_actual)
filtered_predicted = predictions[indices_to_filter_for]

mse_filtered = loss_fct(filtered_actual, filtered_predicted)
baseline_filtered = loss_fct(filtered_actual, torch.full_like(filtered_actual, filtered_actual_mean))
loss_fct_aux = torch.nn.MSELoss(reduction='none')
variance = torch.var(loss_fct_aux(filtered_actual, torch.full_like(filtered_actual, filtered_actual_mean)))
r_squared = hf.compute_r2_torch(preds=filtered_predicted, targets=filtered_actual)
print(baseline_filtered)
print(mse_filtered)
print(r_squared)
loss_fct_aux = torch.nn.MSELoss(reduction='none')
variance = torch.var(loss_fct_aux(filtered_actual, torch.full_like(filtered_actual, filtered_actual_mean)))
print(f'variance: {variance}')


# actual_mean = torch.mean(actual_vals)
# variance = torch.var(loss_fct_aux(actual_vals, torch.full_like(actual_vals, actual_mean)))
# print(f'Test Loss: {test_loss}')
# print(f'Baseline Loss: {baseline_loss}')
# print(f'r_squared: {r_squared}')
# print(f'variance: {variance}')
# # print("Number of values where 'og_capacity_reduction' is not 0 with tolerance 1e-3:", num_values_not_zero)

# gdf_in_meters = gdf_with_og_values.to_crs("EPSG:32633")
# gdf_in_meters.length

# tolerance = 1e-3
# gdf_with_capacity_reduction = gdf_in_meters[abs(gdf_in_meters['og_capacity_reduction']) > tolerance]
# gdf_with_capacity_reduction['length'] = gdf_with_capacity_reduction.length
# total_length = gdf_with_capacity_reduction['length'].sum()/1000

tensor(12.1289, device='cuda:0')
tensor(5.4791, device='cuda:0')
tensor(0.5483, device='cuda:0')
variance: 785.4876098632812


In [9]:
# loss_fct = torch.nn.MSELoss()

# test_loss, r_squared, actual_vals, predictions, baseline_loss = validate_trained_model(model, test_set_loader.dataset, loss_fct, device)
# print(f'Test Loss: {test_loss}')
# print(f'r_squared: {r_squared}')
# print(f'baseline_loss: {baseline_loss}')

# print(f'actual_vals shape: {len(actual_vals)}')
# print(f'predictions shape: {len(predictions)}')



In [10]:
# Function to filter list of tensors
# def filter_tensors_by_indices(tensor_list, indices):
#     return [tensor[indices] for tensor in tensor_list]

In [11]:
# Ensure actual_vals and predictions are tensors
# if isinstance(actual_vals, list):
#     actual_vals = torch.tensor(actual_vals)
# if isinstance(predictions, list):
#     predictions = torch.tensor(predictions)
# THIS IS FOR WHOLE TEST SET ! 


# # Filter actual and predicted values
# filtered_actual = filter_tensors_by_indices(actual_vals, indices_to_filter_for)
# filtered_predicted = filter_tensors_by_indices(predictions, indices_to_filter_for)

# # Concatenate filtered tensors for computing metrics
# filtered_actual_concat = torch.cat(filtered_actual)
# filtered_predicted_concat = torch.cat(filtered_predicted)

# filtered_actual_mean = torch.mean(filtered_actual_concat)
# # filtered_actual = actual_vals[indices_to_filter_for]
# # filtered_actual_mean = torch.mean(filtered_actual)
# # filtered_predicted = predictions[indices_to_filter_for]

# mse_filtered = loss_fct(filtered_actual_concat, filtered_predicted_concat)
# baseline_filtered = loss_fct(filtered_actual_concat, torch.full_like(filtered_actual_concat, filtered_actual_mean))
# r_squared = compute_r2_torch(preds=filtered_predicted_concat, targets=filtered_actual_concat)
# loss_fct_aux = torch.nn.MSELoss(reduction='none')
# variance = torch.var(loss_fct_aux(filtered_actual_concat, torch.full_like(filtered_actual_concat, filtered_actual_mean)))

# print(f'variance: {variance}')
# print(f'Baseline Loss: {baseline_filtered}')
# print(f'Test Loss: {mse_filtered}')
# print(f'r_squared: {r_squared}')


In [12]:
indices_to_filter_for = no_filter

In [13]:
# # gdf = hf.data_to_geodataframe(data=test_set_loader.dataset[0], original_gdf=test_data, predicted_values=predictions)
# # gdf_with_og_values = hf.map_to_original_values(input_gdf=gdf, scaler_x=scaler_x)
# # gdf_with_og_values.crs = "EPSG:2154"
# # gdf_with_og_values.to_crs("EPSG:4326", inplace=True)

# gdf_in_meters = gdf_with_og_values.to_crs("EPSG:32633")
# gdf_in_meters.length

# tolerance = 1e-3
# gdf_with_capacity_reduction = gdf_in_meters.loc[indices_to_filter_for]
# gdf_with_capacity_reduction['length'] = gdf_with_capacity_reduction.length
# total_length = gdf_with_capacity_reduction['length'].sum()/1000
# print(total_length)

In [15]:
gdf_in_meters = gdf_with_og_values.to_crs("EPSG:32633")
gdf_in_meters.length

tolerance = 1e-3
gdf_with_capacity_reduction = gdf_in_meters[abs(gdf_in_meters['og_capacity_reduction']) > tolerance]
gdf_with_capacity_reduction['length'] = gdf_with_capacity_reduction.length
total_length = gdf_with_capacity_reduction['length'].sum()/1000

/opt/anaconda3/envs/chenhao-gnn/lib/python3.10/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [16]:
len(gdf_with_capacity_reduction)

4492